In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from companies.models import Company
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [29]:
from django.db.models import Count, F, Value, Avg, Min, Max
from django.db.models.functions import Length, Upper, Concat
from django.db.models.lookups import GreaterThan

# อันนี้ลองเฉย ๆ อยากเห็นหน้าตา
from django.forms.models import model_to_dict

In [5]:
Company.objects.filter(num_employees__gt=F("num_chairs"))

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

In [6]:
Company.objects.filter(num_employees__gt=F("num_chairs") * 2)

<QuerySet [<Company: Company CCC>]>

In [7]:
Company.objects.filter(num_employees__gt=F("num_chairs") + F("num_tables"))

<QuerySet [<Company: Company CCC>]>

In [27]:
company = (
    Company.objects.filter(num_employees__gt=F("num_chairs"))
    .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
    .first()
)

# annotate is create new field

In [9]:
company.num_employees

50

In [10]:
company.num_chairs

30

In [11]:
company.chairs_needed

20

In [30]:
# อันนี้ลองเฉย ๆ อยากเห็นหน้าตา

# from django.forms.models import model_to_dict

print(model_to_dict(company))


{'id': 2, 'name': 'Company BBB', 'ticker': 'BBB', 'num_employees': 50, 'num_tables': 20, 'num_chairs': 30}


In [38]:
# import Concat ก่อนด้วย

companies = Company.objects.annotate(
    full_name = Concat("name", Value(" ("), "ticker", Value(")"))
)

for com in companies:
    print(com.full_name)

Company AAA (AAA\\)
Company BBB (BBB)
Company CCC (CCC)


In [40]:
# comps = Company.objects.filter
#     num_employees

# aggregate expression

In [5]:
Book.objects.filter(publisher__name="Penguin Books").count()

20

In [9]:
# aggregate avg
Book.objects.aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.7018644067796610')}

In [13]:
# filter then aggregate then avg
Book.objects.filter(publisher__name="Penguin Books").aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.6150000000000000')}

In [16]:
res = Book.objects.aggregate(
    price_avg=Avg("price", default=0),
    price_min=Min("price", default=0),
    price_max=Max("price", default=0)
)

res

{'price_avg': Decimal('9.7018644067796610'),
 'price_min': Decimal('5.99'),
 'price_max': Decimal('14.99')}

In [21]:
# pubs = Publisher.objects.annotate(num_books=Count("book"))
# group by อะไร เขียน group.objects... Count 

pubs = Publisher.objects.annotate(avg_price_books=Avg("book__price"))

for pub in pubs:
#     print(f"PUB: {pub.name} => BOOK NUM: {pub.num_books}")
    print(f"PUB: {pub.name} => BOOK NUM: {pub.avg_price_books}")
    

PUB: Oxford University Press => BOOK NUM: 9.7464102564102564
PUB: Penguin Books => BOOK NUM: 9.6150000000000000


# filter

In [23]:
from django.db.models import Q
above = Publisher.objects.annotate(above_4=Count("book", filter=Q(book__rating__gt=4)))
below = Publisher.objects.annotate(below_4=Count("book", filter=Q(book__rating__lte=4)))

In [24]:
above[0].above_4

16

In [25]:
below[0].below_4

4

In [26]:
pubs = Publisher.objects.annotate(num_books=Count("book")).order_by("-num_books")[:5]
pubs[0].num_books

39